In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense,MaxPooling2D,Conv2D,Dropout, BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [19]:
train_data = tf.keras.utils.image_dataset_from_directory(
    'grape',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=42,
    # validation_split=0.2,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    # data_format=None,
    verbose=True,
)

Found 4510 files belonging to 4 classes.


In [20]:
train_size = 0.8
len(train_data)*train_size

112.80000000000001

In [21]:
train_ds = train_data.take(102)
len(train_ds)

102

In [22]:
test_ds = train_data.skip(102)
len(test_ds)

39

In [23]:
val_size=0.1
len(train_data)*val_size

14.100000000000001

In [24]:
val_ds = test_ds.take(18)
len(val_ds)

18

In [25]:
test_ds = test_ds.skip(18)
len(test_ds)

21

In [26]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [27]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(train_data)

In [28]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
#shuffle the data to get good acc

In [29]:
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
])

In [30]:
train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
from keras.applications import ResNet50
conv_base = ResNet50(
           weights = 'imagenet',
           include_top = False,
           input_shape = (128,128,3)
)


out = conv_base.output
# out  = GlobalAveragePooling2D()(out)
out = Flatten()(out)
out = Dense(512, activation='relu')(out)
out = Dropout(0.2)(out)
out = Dense(256, activation='relu')(out)
predictions = Dense(4, activation='softmax')(out)
model = Model(inputs=conv_base.input, outputs=predictions)

In [36]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [37]:
model.summary()

Model: "functional_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)    │ (None, 128, 128, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_pad (ZeroPadding2D)     │ (None, 134, 134, 3)       │               0 │ input_layer_5[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_conv (Conv2D)           │ (None, 64, 64, 64)        │           9,472 │ conv1_pad[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_bn (BatchNormalization) │ (None, 64, 64, 64)        │             256 │ conv1_conv[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_relu (Activation)       │ (None, 64, 64, 64)        │               0 │ conv1_bn[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pool1_pad (ZeroPadding2D)     │ (None, 66, 66, 64)        │               0 │ conv1_relu[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pool1_pool (MaxPooling2D)     │ (None, 32, 32, 64)        │               0 │ pool1_pad[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_conv (Conv2D)  │ (None, 32, 32, 64)        │           4,160 │ pool1_pool[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_bn             │ (None, 32, 32, 64)        │             256 │ conv2_block1_1_conv[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_relu           │ (None, 32, 32, 64)        │               0 │ conv2_block1_1_bn[0][0]    │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_conv (Conv2D)  │ (None, 32, 32, 64)        │          36,928 │ conv2_block1_1_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_bn             │ (None, 32, 32, 64)        │             256 │ conv2_block1_2_conv[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_relu           │ (None, 32, 32, 64)        │               0 │ conv2_block1_2_bn[0][0]    │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_0_conv (Conv2D)  │ (None, 32, 32, 256)       │          16,640 │ pool1_pool[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_3_conv (Conv2D)  │ (None, 32, 32, 256)       │          16,640 │ conv2_block1_2_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 40,497,796 (154.49 MB)

 Trainable params: 40,444,676 (154.28 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [38]:
history = model.fit(
    x=train_ds,
    batch_size=32,
    validation_data=val_ds,
    verbose=1,
    epochs=7,
)

Epoch 1/7
112/112 ━━━━━━━━━━━━━━━━━━━━ 366s 3s/step - accuracy: 0.7948 - loss: 0.5812 - val_accuracy: 0.9397 - val_loss: 0.1929
Epoch 2/7
112/112 ━━━━━━━━━━━━━━━━━━━━ 287s 3s/step - accuracy: 0.9789 - loss: 0.0612 - val_accuracy: 0.9911 - val_loss: 0.0336
Epoch 3/7
112/112 ━━━━━━━━━━━━━━━━━━━━ 284s 3s/step - accuracy: 0.9876 - loss: 0.0380 - val_accuracy: 0.9554 - val_loss: 0.1242
Epoch 4/7
112/112 ━━━━━━━━━━━━━━━━━━━━ 301s 3s/step - accuracy: 0.9871 - loss: 0.0392 - val_accuracy: 0.9732 - val_loss: 0.0560
Epoch 5/7
112/112 ━━━━━━━━━━━━━━━━━━━━ 301s 3s/step - accuracy: 0.9915 - loss: 0.0336 - val_accuracy: 0.9888 - val_loss: 0.0261
Epoch 6/7
112/112 ━━━━━━━━━━━━━━━━━━━━ 289s 3s/step - accuracy: 0.9926 - loss: 0.0198 - val_accuracy: 0.9888 - val_loss: 0.0468
Epoch 7/7
112/112 ━━━━━━━━━━━━━━━━━━━━ 282s 3s/step - accuracy: 0.9941 - loss: 0.0163 - val_accuracy: 0.9754 - val_loss: 0.1366


In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [40]:
model.evaluate(test_ds)

15/15 ━━━━━━━━━━━━━━━━━━━━ 11s 576ms/step - accuracy: 0.9706 - loss: 0.1657


[0.15006297826766968, 0.9750000238418579]

In [41]:
model.save("grape_model.keras")

In [42]:
import json
with open("grape_hist01.json","w") as f:
    json.dump(history.history , f)

In [43]:
class_names = train_data.class_names
class_names

['Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy']